## Major Assignment 1 
### Environmental Analysis Tools (ENEN90032) 
#### Authors: Olivia Borgstroem (1049030), Navindu de Silva (1084196), Robert Strong (1080043) 
#### Created: 22/08/2022
#### Last Edited: 22/08/2022

This is a notebook which complete the tasks outlined in 2022_ENEN90032_Assignment_01.pdf

Git Hub Address: https://github.com/robertstrong10/EAT_GP01.git

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math

Task 1 Exploratory Data Analysis - Meteorological Datasets

Atmospheric CO2 Concentration during Global Forced Confinement by COVID-19ced Confinement by COVID-19

Newcomb-Michelson Velocity of Light Experiments

In [2]:
light_df = pd.read_csv('NewcombLight.txt', header=None)
distance = 7442 # m at sea level
wikipedia_speed = 299792458 # m/s accessed from https://en.wikipedia.org/wiki/Speed_of_light

light_df = distance/ light_df
light_df.columns = ["speed"] 
light_mean = np.mean(light_df.speed)


# T-test
light_t = stats.ttest_1samp(light_df, light_mean, alternative='two-sided', )
print("Sample Mean = {} m/s".format(light_mean))
print("T-test pvalue = {}".format(light_t.pvalue[0])) 
## pvalue is very large (~= 1) hence accept that the null hypothesis (that mean is correct)


# Bootstrap
light_b_data = (light_df,)
light_bootstrap_ci = stats.bootstrap(light_b_data, np.mean, confidence_level=0.99, random_state=1, method='percentile')
print("99% CI from Bootstrap: ({}, {})".format(light_bootstrap_ci.confidence_interval[0][0], 
                                               light_bootstrap_ci.confidence_interval[1][0]))

print("Speed of Light (Wikipedia, 2022) = {} m/s".format(wikipedia_speed))
print("Percentage Error Sample Mean and Wikipedia = {}%".format(abs((light_mean-wikipedia_speed)/light_mean *100)))
print("Percentage Error q0.5 Bootstrap and Wikipedia = {}%".format(
    abs(((light_bootstrap_ci.confidence_interval[0][0] +
          light_bootstrap_ci.confidence_interval[1][0])/2-wikipedia_speed)
        /(light_bootstrap_ci.confidence_interval[0][0] 
          + light_bootstrap_ci.confidence_interval[1][0])/2 *100)))


Sample Mean = 299763868.09810716 m/s
T-test pvalue = 1.0
99% CI from Bootstrap: (299731082.5421092, 299814979.065918)
Speed of Light (Wikipedia, 2022) = 299792458 m/s
Percentage Error Sample Mean and Wikipedia = 0.00953747430410256%
Percentage Error q0.5 Bootstrap and Wikipedia = 0.0016201587526306926%


Space Shuttle O-Ring Failures

In [6]:
temps = pd.read_excel("O_Ring_Data.XLS")
temps

,INCIDENTS,LAUNCH
0,1,COOL
1,1,COOL
2,1,COOL
3,3,COOL
4,0,WARM
5,0,WARM
6,0,WARM
7,0,WARM
8,0,WARM
9,0,WARM


Cloud Seeding Experiment

Exploratory Data Analysis and Linear Regression (RS)

In [ ]:
Ndata = pd.read_csv('/Users/RobStrong/Desktop/EAT/Ass1/Q_TKN_data.csv')
Ndata = Ndata.rename(columns={"Q (mm/d)":"Q", "TKN (mg/L)":"TKN"})

N_person = stats.pearsonr(Ndata.Q, Ndata.TKN)
N_spearman = stats.spearmanr(Ndata.Q, Ndata.TKN)
print("Pearson Correlation Coefficient = ",N_person.statistic)
print("Spearman’s Rank Correlation Coefficient = ", N_spearman.correlation)

N_log_person = stats.pearsonr(np.log(Ndata.Q), np.log(Ndata.TKN))
N_log_spearman = stats.spearmanr(np.log(Ndata.Q), np.log(Ndata.TKN))
print("Logged Data Pearson Correlation Coefficient = ",N_log_person.statistic)
print("Logged Data Spearman’s Rank Correlation Coefficient = ", N_log_spearman.correlation)


fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10,4))
ax1.scatter(Ndata.Q, Ndata.TKN, c='r')
ax1.set_xlabel('Q')
ax1.set_ylabel('TKN')
ax2.scatter(np.log(Ndata.Q), np.log(Ndata.TKN), c='b')
ax2.set_xlabel('log(Q)')
ax2.set_ylabel('log(TKN)')
plt.suptitle('Raw vs Log')
plt.show()


# Linear Regression (using log transformed data)
N_slope, N_intercept, N_r, N_p, N_se = stats.linregress(np.log(Ndata.Q), np.log(Ndata.TKN))

# predicting TKN at Q = 2mm 
TKN_log_predicted = (N_slope * np.log(2)) + N_intercept # log(y) = m*log(x) + c
TKN_predicted = np.exp(TKN_log_predicted) # transforming back to original domain
TKN_predicted

# 95% CI on Conditional Mean and Prediction

plt.figure()
plt.scatter(np.log(Ndata.Q), np.log(Ndata.TKN), c='b')
plt.plot(np.log(Ndata.Q), N_slope*np.log(Ndata.Q)+ N_intercept, 'k-')
plt.xlabel('log(Q)')
plt.ylabel('log(TKN)')
plt.text(-5,0.8, s="r = " + str(np.round(N_r,4)))
plt.text(-5,0.7, s="log(TKN) = " + str(np.round(N_slope,4)) + "* log(Q) + " + str(np.round(N_intercept,4)))
plt.show()


In [ ]:
N = len(Ndata)
X = np.log(Ndata.Q)
meanX = np.mean(X)
Yhat = N_slope * X + N_intercept

SE_CI_95 = N_se * np.sqrt(1/N + ((X - meanX)**2 / np.sum((X-meanX)**2)))
SE_PredCI = N_se  * np.sqrt( 1 + 1/N + ((X - meanX)**2 / np.sum((X-meanX)**2)))

# Plot the regression line with 95% CI of predicted values and 95% CI of conditional mean
import scipy.stats as stats
t_dist  = stats.t(N-2)

# Add regression line to scatter plot 
plt.figure(figsize=(8,8))
plt.scatter(np.log(Ndata.Q), np.log(Ndata.TKN), s = 80, color='orange', alpha=0.6, edgecolor='red', linewidth=1)
plt.xlabel('log(Q)')
plt.ylabel('log(TKN)')
plt.title('Linear Regression of Q vs TKN for Curdies River at Curdie')
plt.plot(X, Yhat, lw = 1, c = 'k', label = 'Regression line') #regression line plotting from sklearn 

# Prediction CI
plt.plot(X, Yhat + t_dist.ppf(0.975)*SE_PredCI, c = 'navy', lw=1, label = '95% CI of Predictions') #upper limit
plt.plot(X, Yhat + t_dist.ppf(0.025)*SE_PredCI, c = 'navy', lw=1, label = None) #lower limit

# Conditional mean CI
plt.plot(X, Yhat + t_dist.ppf(0.975)*SE_CI_95, c = 'teal', lw=1, label = '95% CI of Conditinonal Mean') #upper limit
plt.plot(X, Yhat + t_dist.ppf(0.025)*SE_CI_95, c = 'teal', lw=1, label = None) #lower limit
plt.legend(loc = 0, fontsize=12)
plt.show()

In [ ]:
count = 0
predictions_upper = Yhat + t_dist.ppf(0.975)*SE_PredCI
predictions_lower = Yhat + t_dist.ppf(0.025)*SE_PredCI


for i in range(len(Ndata)):
    if (np.log(Ndata.TKN[i]) > predictions_lower[i] and np.log(Ndata.TKN[i]) < predictions_upper[i]):
        count += 1

count # = 6

Atmospheric CO2 Concentration during Global Forced Confinement by COVID-19ced Confinement by COVID-19